In [1]:
import pandas as pd
from datasets import load_from_disk

test_dataset = load_from_disk('/opt/ml/data/test_dataset')
test_dataset = test_dataset['validation']

test_data = []

for i, row in enumerate(test_dataset):
    test_data.append(row)

test_df = pd.DataFrame(test_data)
test_df['test_id_no'] = test_df['id'].apply(lambda x: x.split('-')[-1])
#test_df.head()

# print(len(test_df))               # 600
# print(len(test_df[['question']])) # 600

def get_klue_df(df, data_source):
    data_objs = {
        'version': [],
        'title': [],
        'context': [],
        'question': [],
        'answer_text': [],
        'answer_start': [],
        'question_type': [],
        'is_impossible': [],
        'guid': [],
        'klue_id_no': [],
        'news_category': [],
        'source': [],
        'data_source': []
    }

    for i, row in df.iterrows():
        data = row['data']
        qas = data['paragraphs'][0]['qas']
        for q in qas:
            if len(q['answers']) == 0:
                continue
            data_objs['version'].append(row['version'])
            data_objs['title'].append(data['title'])
            data_objs['context'].append(data['paragraphs'][0]['context'])
            data_objs['question'].append(q['question'])
            answer_text_list = []
            answer_start_list = []
            for ans in q['answers']:
                answer_text_list.append(ans['text'])
                answer_start_list.append(ans['answer_start'])
            # data_objs['answer_text'].append(q['answers'][0]['text'])
            # data_objs['answer_start'].append(q['answers'][0]['answer_start'])
            data_objs['answer_text'].append(answer_text_list)
            data_objs['answer_start'].append(answer_start_list)        
            data_objs['question_type'].append(q['question_type'])
            data_objs['is_impossible'].append(q['is_impossible'])
            data_objs['guid'].append(q['guid'])
            data_objs['klue_id_no'].append(q['guid'].split('_')[-1])
            data_objs['news_category'].append(data['news_category'])
            data_objs['source'].append(data['source'])
            data_objs['data_source'].append(data_source)

    return pd.DataFrame(data_objs)

klue_train_df = get_klue_df(pd.read_json('./klue-mrc-v1.1_train.json'), 'train')
klue_dev_df = get_klue_df(pd.read_json('./klue-mrc-v1.1_dev.json'), 'dev')

# print(len(klue_train_df))                 # 11167
# print(len(klue_train_df[['question']]))   # 11167
# print(len(klue_dev_df))                   # 3846
# print(len(klue_dev_df[['question']]))     # 3846

klue_df = pd.concat([klue_train_df, klue_dev_df])
# print(len(klue_df))                 # 15013
# print(len(klue_df[['question']]))   # 15013
#klue_df.head()

merge_df = pd.merge(test_df, klue_df, how='outer', on='question')
merge_df['id'].fillna('miss', inplace=True)
merge_df['answer_text'].fillna('none', inplace=True)
merge_df = merge_df.loc[merge_df['id'] != 'miss', :]
merge_df = merge_df.loc[:, ['id', 'question', 'context', 'answer_text','data_source', 'test_id_no', 'klue_id_no']]
merge_df['id_type'] = merge_df['id'].apply(lambda x: x.split('-')[1])
merge_df['answer_cnt'] = merge_df['answer_text'].apply(lambda x: len(x) if type(x) == list else 0)
merge_df['answer_yn'] = merge_df['answer_text'].apply(lambda x: True if x != 'none' else False)
merge_df

,id,question,context,answer_text,data_source,test_id_no,klue_id_no,id_type,answer_cnt,answer_yn
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간...,[사락사라],dev,000653,00653,1,1,True
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...,[냉전],dev,001113,01113,1,1,True
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ...",[빌헬름 미클라스],train,002191,02191,0,1,True
3,mrc-0-003951,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?",NaN,none,NaN,003951,NaN,0,0,False
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...,[프랑스],dev,001272,01272,1,1,True
...,...,...,...,...,...,...,...,...,...,...
595,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,...",[재단 X],train,002989,02989,0,1,True
596,mrc-0-001804,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?,NaN,none,NaN,001804,NaN,0,0,False
597,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",콜드게임(called game)이란 야구나 소프트볼 등의 운동경기에서 심판에 의하여...,[서스펜디드 게임],train,003411,03411,0,1,True
598,mrc-0-003436,제2캐나다기갑여단이 상륙한 곳은?,주노 해변\n\n주노 해변은 쿠르쇨르메르의 양쪽으로 뻗은 5마일 정도의 해변이었다....,[주노 해변],train,003436,03436,0,1,True


In [103]:
merge_df['answer_yn'].value_counts()

True     475
False    125
Name: answer_yn, dtype: int64

In [105]:
merge_df.to_csv('./test_dataset_result.csv', index=False)

In [2]:
test_result_df = merge_df[['id', 'question', 'answer_text', 'answer_cnt', 'answer_yn']]
test_result_df

,id,question,answer_text,answer_cnt,answer_yn
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,[사락사라],1,True
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,[냉전],1,True
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,[빌헬름 미클라스],1,True
3,mrc-0-003951,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?",none,0,False
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,[프랑스],1,True
...,...,...,...,...,...
595,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,[재단 X],1,True
596,mrc-0-001804,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?,none,0,False
597,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",[서스펜디드 게임],1,True
598,mrc-0-003436,제2캐나다기갑여단이 상륙한 곳은?,[주노 해변],1,True


In [110]:
test_result_df.to_csv('./test_dataset_with_answer.csv', index=False)

In [111]:
answer_df = test_result_df.loc[test_result_df['answer_yn'] == True, :]
answer_df

,id,question,answer_text,answer_cnt,answer_yn
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,[사락사라],1,True
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,[냉전],1,True
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,[빌헬름 미클라스],1,True
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,[프랑스],1,True
5,mrc-1-000993,단공류가 일반 포유류와 다르다는 것을 알 수 있는 신체 부위는?,[뒷다리],1,True
...,...,...,...,...,...
594,mrc-0-000651,디즈니랜드에서 개인회원을 뽑기 시작한 년도는?,"[1967년, 1967]",2,True
595,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,[재단 X],1,True
597,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",[서스펜디드 게임],1,True
598,mrc-0-003436,제2캐나다기갑여단이 상륙한 곳은?,[주노 해변],1,True


In [113]:
answer_df.loc[answer_df['answer_cnt'] == 1, :]

,id,question,answer_text,answer_cnt,answer_yn
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,[사락사라],1,True
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,[냉전],1,True
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,[빌헬름 미클라스],1,True
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,[프랑스],1,True
5,mrc-1-000993,단공류가 일반 포유류와 다르다는 것을 알 수 있는 신체 부위는?,[뒷다리],1,True
...,...,...,...,...,...
592,mrc-0-004394,《그리스 비극 시대의 철학》은 누구의 저서인가?,[프리드리히 니체],1,True
593,mrc-0-004335,중국 공산당원과 가족의 미국 입국 금지 방안 검토와 관련된 마이크 폼페이오 장관의 ...,[뉴욕타임스],1,True
595,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,[재단 X],1,True
597,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",[서스펜디드 게임],1,True
